# CIFAR-10 Robustness and Explainability Benchmark
This notebook benchmarks ResNet-18, ViT Transformer on CIFAR-10 under various corruptions.  

# Imports and Installation

In [ ]:
!pip install timm torchcam --quiet

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import timm
from torchcam.methods import SmoothGradCAMpp  # for ViT and ResNet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

# CIFAR10 Classes and Sample Images

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
print("CIFAR-10 classes:", trainset.classes)
plt.figure(figsize=(8,2))
for i in range(6):
    img, label = trainset[i]
    plt.subplot(1,6,i+1)
    plt.imshow(img)
    plt.title(trainset.classes[label])
    plt.axis('off')
plt.show()

# Defining the Transforms

* clean
* horizontal flip
* rotation
* blur
* brightness
* gaussian noise

In [ ]:
# For training (with data augmentation)
def make_train_transform(img_size):
    return T.Compose([
        T.Resize(img_size),
        T.RandomHorizontalFlip(),
        T.RandomCrop(img_size, padding=4),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])

# For testing and robustness evaluation
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.1):
        self.mean = mean
        self.std = std
    def __call__(self, tensor):
        return tensor + torch.randn_like(tensor) * self.std + self.mean

def make_transforms(img_size):
    return {
        'Clean': T.Compose([T.Resize(img_size), T.ToTensor(), T.Normalize(mean, std)]),
        'Horizontal Flip': T.Compose([T.Resize(img_size), T.RandomHorizontalFlip(1.0), T.ToTensor(), T.Normalize(mean, std)]),
        'Rotation': T.Compose([T.Resize(img_size), T.RandomRotation(30), T.ToTensor(), T.Normalize(mean, std)]),
        'Blur': T.Compose([T.Resize(img_size), T.GaussianBlur(3), T.ToTensor(), T.Normalize(mean, std)]),
        'Brightness': T.Compose([T.Resize(img_size), T.ColorJitter(brightness=0.5), T.ToTensor(), T.Normalize(mean, std)]),
        'Gaussian Noise': T.Compose([T.Resize(img_size), T.ToTensor(), AddGaussianNoise(0., 0.15), T.Normalize(mean, std)]),
    }

# Training and Evaluation Functions

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        if outputs.dim() == 3:
            outputs = outputs.mean(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, 100.0 * correct / total

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.dim() == 3:
                outputs = outputs.mean(1)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

# Defining the Models

In [ ]:
models = [
    ('ResNet-18', lambda: torchvision.models.resnet18(weights='IMAGENET1K_V1'), 32),
    ('ViT-Small', lambda: timm.create_model('vit_small_patch16_224', pretrained=True), 224),
]

def adjust_model(model):
    if hasattr(model, 'fc'):  # ResNet
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif hasattr(model, 'head'):  # ViT
        model.head = nn.Linear(model.head.in_features, 10)
    return model

# Training Loop for ResNet18 and ViT Models

In [ ]:
epochs = 10    
batch_size = 128

for model_name, model_func, img_size in models:
    print(f"\n=== Training {model_name} on CIFAR-10 ===")
    model = model_func()
    model = adjust_model(model)
    model = model.to(device)

    # Datasets and loaders
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=make_train_transform(img_size))
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=make_transforms(img_size)['Clean'])
    testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(model, trainloader, criterion, optimizer, device)
        test_acc = evaluate(model, testloader)
        print(f"Epoch {epoch+1}/{epochs} - Train Acc: {train_acc:.2f}% - Test Acc: {test_acc:.2f}%")
    
    torch.save(model.state_dict(), f"{model_name}_cifar10.pth")
    print(f"Saved {model_name} trained weights.\n")

# Robustness Benchmarking

In [ ]:
import pandas as pd

models = [
    ('ResNet-18', lambda: torchvision.models.resnet18(weights=None), 32),
    ('ViT-Small', lambda: timm.create_model('vit_small_patch16_224', pretrained=False), 224),
]

model_paths = {
    "ResNet-18": "/kaggle/input/resnet18_cifar10/pytorch/default/1/ResNet-18_cifar10.pth",
    "ViT-Small": "/kaggle/input/vit-cifar10/pytorch/default/1/ViT-Small_cifar10.pth"
}

results = []

for model_name, model_func, img_size in models:
    print(f"\nEvaluating {model_name} robustness (input size {img_size})...")
    model = model_func()
    model = adjust_model(model)
    state_dict = torch.load(model_paths[model_name], map_location=device)
    model.load_state_dict(state_dict)
    model = model.to(device)
    model.eval()
    curr_transforms = make_transforms(img_size)  
    testloaders = {}
    for name, transform in curr_transforms.items():
        dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
        testloaders[name] = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=False, num_workers=2)
    model_results = {'Model': model_name}
    for corr_name, loader in testloaders.items():
        acc = evaluate(model, loader)
        model_results[corr_name] = acc
        print(f"{corr_name}: {acc:.2f}%")
    results.append(model_results)

df = pd.DataFrame(results)
df.set_index('Model', inplace=True)
display(df.style.background_gradient(cmap='Blues'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

# Get all transform types
all_transforms = make_transforms(32)
n_transforms = len(all_transforms)
n_images = 8  # images per row

plt.figure(figsize=(2*n_images, 2*n_transforms))

for idx, (transform_name, transform) in enumerate(all_transforms.items()):
    dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=n_images, shuffle=True)
    images, labels = next(iter(loader))
    for j in range(n_images):
        plt_idx = idx * n_images + j + 1
        ax = plt.subplot(n_transforms, n_images, plt_idx)
        img_np = images[j].cpu().numpy().transpose(1,2,0)
        img_np = img_np * np.array(std) + np.array(mean)
        img_np = np.clip(img_np, 0, 1)
        ax.imshow(img_np)
        ax.set_xticks([])
        ax.set_yticks([])
        if j == 0:
            ax.set_ylabel(transform_name, fontsize=11, rotation=0, labelpad=60, va='center')
        else:
            ax.set_ylabel('')
        ax.set_title(f"{labels[j].item()}", fontsize=8)
plt.suptitle("Transforms Applied for the Images", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()